In [1]:
from langchain_core.runnables import  RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from langchain_core.output_parsers import StrOutputParser
from langchain_community.graphs import Neo4jGraph
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.chat_models import ChatOllama
from langchain_experimental.graph_transformers import LLMGraphTransformer
from neo4j import GraphDatabase
from yfiles_jupyter_graphs import GraphWidget
from langchain_community.vectorstores import Neo4jVector
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars
from langchain_ollama import OllamaEmbeddings
from langchain_ollama import ChatOllama
import os
from langchain_experimental.llms.ollama_functions import OllamaFunctions
from neo4j import  Driver

from dotenv import load_dotenv

load_dotenv()

True

In [2]:
graph = Neo4jGraph()

/tmp/ipykernel_15238/267675202.py:1: LangChainDeprecationWarning: The class `Neo4jGraph` was deprecated in LangChain 0.3.8 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-neo4j package and should be used instead. To use it run `pip install -U :class:`~langchain-neo4j` and import as `from :class:`~langchain_neo4j import Neo4jGraph``.
  graph = Neo4jGraph()


In [4]:
loader = TextLoader(file_path="Dummy_Data.txt")
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=250, chunk_overlap=24)
documents = text_splitter.split_documents(documents=docs)

In [ ]:

llm = ChatOllama(model="llama3.2:latest", temperature=0)

llm_transformer = LLMGraphTransformer(llm=llm)

graph_documents = llm_transformer.convert_to_graph_documents(documents)

In [6]:
graph_documents[0]

GraphDocument(nodes=[Node(id="Amico'S Family", type='Family', properties={}), Node(id='Love', type='Emotion', properties={}), Node(id='Tradition', type='Concept', properties={})], relationships=[Relationship(source=Node(id="Amico'S Family", type='Family', properties={}), target=Node(id='Love', type='Emotion', properties={}), type='RELATED_TO', properties={}), Relationship(source=Node(id="Amico'S Family", type='Family', properties={}), target=Node(id='Tradition', type='Concept', properties={}), type='RELATED_TO', properties={})], source=Document(metadata={'source': 'Dummy_Data.txt'}, page_content='1. The Story of Amico’s Family: A Legacy of Love and Tradition'))

In [7]:
graph.add_graph_documents(
    graph_documents,
    baseEntityLabel=True,
    include_source=True
)

In [8]:
def showGraph():
    driver = GraphDatabase.driver(
        uri = os.environ["NEO4J_URI"],
        auth = (os.environ["NEO4J_USERNAME"],
                os.environ["NEO4J_PASSWORD"]))

    session = driver.session()

    widget = GraphWidget(graph = session.run("MATCH (s)-[r :!MENTIONS]->(t) RETURN s,r,t").graph())
    widget.node_label_mapping = 'id'
    return widget

showGraph()

GraphWidget(layout=Layout(height='800px', width='100%'))

In [9]:
embeddings = OllamaEmbeddings(
    model="nomic-embed-text:latest",
)

vector_index = Neo4jVector.from_existing_graph(
    embeddings,
    search_type="hybrid",
    node_label="Document",
    text_node_properties=["text"],
    embedding_node_property="embedding"
)
vector_retriever = vector_index.as_retriever()

In [10]:
driver = GraphDatabase.driver(
        uri = os.environ["NEO4J_URI"],
        auth = (os.environ["NEO4J_USERNAME"],
                os.environ["NEO4J_PASSWORD"]))

def create_fulltext_index(tx):
    query = '''
    CREATE FULLTEXT INDEX `fulltext_entity_id` 
    FOR (n:__Entity__) 
    ON EACH [n.id];
    '''
    tx.run(query)

# Function to execute the query
def create_index():
    with driver.session() as session:
        session.execute_write(create_fulltext_index)
        print("Fulltext index created successfully.")

# Call the function to create the index
try:
    create_index()
except:
    pass

# Close the driver connection
driver.close()

Fulltext index created successfully.


In [11]:

class Entities(BaseModel):
    """Identifying information about entities."""

    names: list[str] = Field(
        ...,
        description="All the person, organization, or business entities that "
        "appear in the text",
    )

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are extracting organization and person entities from the text.",
        ),
        (
            "human",
            "Use the given format to extract information from the following "
            "input: {question}",
        ),
    ]
)


entity_chain = llm.with_structured_output(Entities)

In [12]:
entity_chain.invoke("Who are Nonna Lucia and Giovanni Caruso?")

Entities(names=['Nonna Lucia', 'Giovanni Caruso'])

In [13]:
def generate_full_text_query(input: str) -> str:
    words = [el for el in remove_lucene_chars(input).split() if el]
    if not words:
        return ""
    full_text_query = " AND ".join([f"{word}~2" for word in words])
    print(f"Generated Query: {full_text_query}")
    return full_text_query.strip()


# Fulltext index query
def graph_retriever(question: str) -> str:
    """
    Collects the neighborhood of entities mentioned
    in the question
    """
    result = ""
    entities = entity_chain.invoke(question)
    for entity in entities.names:
        response = graph.query(
            """CALL db.index.fulltext.queryNodes('fulltext_entity_id', $query, {limit:2})
            YIELD node,score
            CALL {
              WITH node
              MATCH (node)-[r:!MENTIONS]->(neighbor)
              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output
              UNION ALL
              WITH node
              MATCH (node)<-[r:!MENTIONS]-(neighbor)
              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS output
            }
            RETURN output LIMIT 50
            """,
            {"query": entity},
        )
        result += "\n".join([el['output'] for el in response])
    return result

In [14]:
print(graph_retriever("Who is Nonna Lucia?"))

Nonna Lucia - INFLUENCE -> None
Nonna Lucia - TAUGHT -> Amico
Nonna Lucia - IS_A_MENTOR -> Mentor
Nonna Lucia - HAS_TITLE -> The Matriarch
Lucia - RESIDES -> Los Angeles
Lucia - LOCATION_BROUGHT_BY -> Los Angeles
Lucia - INHERITED_TALENT -> Sicily
Lucia - SISTER -> Antonio
Lucia - GRANDMOTHER -> Amico
Baking - RELATED_TO -> Lucia
Bella Vita - SUPPORTS -> LuciaLucia - RESIDES -> Los Angeles
Lucia - LOCATION_BROUGHT_BY -> Los Angeles
Lucia - INHERITED_TALENT -> Sicily
Lucia - SISTER -> Antonio
Lucia - GRANDMOTHER -> Amico
Baking - RELATED_TO -> Lucia
Bella Vita - SUPPORTS -> Lucia
Nonna Lucia - INFLUENCE -> None
Nonna Lucia - TAUGHT -> Amico
Nonna Lucia - IS_A_MENTOR -> Mentor
Nonna Lucia - HAS_TITLE -> The Matriarch


In [15]:
def full_retriever(question: str):
    graph_data = graph_retriever(question)
    vector_data = [el.page_content for el in vector_retriever.invoke(question)]
    final_data = f"""Graph data:
{graph_data}
vector data:
{"#Document ". join(vector_data)}
    """
    return final_data

In [16]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
Use natural language and be concise.
Answer:"""
prompt = ChatPromptTemplate.from_template(template)

chain = (
        {
            "context": full_retriever,
            "question": RunnablePassthrough(),
        }
    | prompt
    | llm
    | StrOutputParser()
)

In [17]:
chain.invoke(input="Who is Nonna Lucia? Did she teach anyone about restaurants or cooking?")

'Nonna Lucia was a mentor and matriarch who taught her grandchildren, including Amico, the art of Sicilian cooking. She also taught others, such as aspiring chefs in Los Angeles, about sustainable cooking practices and the importance of using organic and local ingredients.'